# **Brandify** is an application powered by AI that provides comprehensive recommendations and guidance to society in their branding strategy. In this notebook, we have implemented three AI models to assist our clients in interacting with the app and exploring our services.



This notebook contains three models built will be deployed in our mobile app to provide guidance, advice, and schedules to our teenage customers.

1.   The first model is a Speech-to-Text model. We utilized Transformer's Speech2TextProcessor and Speech2TextForConditionalGeneration.

2.  The second model is an NLP-driven tool developed for businesses to formulate effective branding strategies. It specializes in generating product names and color palettes tailored to enhance brand visibility and appeal within the corporate landscape

3. The third model is a Text-to-Speech model. It takes the output from the second model and transforms it into speech using the SpeechT5Processor, SpeechT5ForTextToSpeech, and SpeechT5HifiGan. Additionally, the set_seed function is implemented to ensure reproducibility in the model's results.

**All three models have been thoroughly tested and function well in the notebook**.

## **1-Speech To Text**

In [2]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
import librosa
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Change the working directory to the folder where your file is located
import os
os.chdir('/content/drive/MyDrive/IA_wave')

# Load the speech-to-text model and processor
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

# Load your own audio file
audio_file_path = 'Record.mp3'
# Adjust chunk size according to your preference and memory constraints
chunk_size_seconds = 10

# Function to transcribe audio chunks
def transcribe_audio_chunk(audio_chunk):
    inputs = processor(audio_chunk, sampling_rate=16000, return_tensors="pt")
    generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
    return transcription

# Load the audio file
audio_signal, sampling_rate = librosa.load(audio_file_path, sr=16000)

# Calculate the number of chunks
chunk_size_samples = chunk_size_seconds * sampling_rate
num_chunks = len(audio_signal) // chunk_size_samples

# Transcribe each chunk and concatenate the transcriptions into a string
transcription_string = ""
for i in range(num_chunks + 1):
    start = i * chunk_size_samples
    end = min((i + 1) * chunk_size_samples, len(audio_signal))
    audio_chunk = audio_signal[start:end]
    transcription_chunk = transcribe_audio_chunk(audio_chunk)
    transcription_string += ' '.join(transcription_chunk) + ' '

print("Transcription:", transcription_string)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Transcription: establish it in twenty ten stylish word in see of her strangely diversed emperor reflecting dynamic designs our vibrant color padded at inclusive iphis letter to fresh and confused individuals seeking eunuch experience 


** 2- NLP Model: This model takes input from teenagers' problems, explores their experiences with anxiety and stress, and examines how these challenges impact their lives. It then offers advice and suggests time management methods to help them navigate through these issues.**






In [3]:
from google.colab import files

uploaded = files.upload()


Saving fin.csv to fin (2).csv


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Load CSV data
data = pd.read_csv("fin.csv", skiprows=[28, 29])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Input   133 non-null    object
 1   Output  133 non-null    object
dtypes: object(2)
memory usage: 2.2+ KB
None
                                               Input  \
0  Our company, XYZ Cosmetics, is launching a new...   
1  Our enterprise, Tech Innovations Inc., is intr...   
2  Our enterprise, Fashion Forward Designs, is la...   
3  Our enterprise, Fresh Eats Inc., is launching ...   
4  Our enterprise, Adventure Gear Co., is launchi...   

                                              Output  
0  Brand the new product line as 'NaturaGlow'. Po...  
1  Position the new smart home device as 'HomeGen...  
2  Brand the new clothing line as 'EcoChic'. Posi...  
3  Position the new plant-based meal kits as 'Gre...  
4  Brand the new outdoor gear line as 'TrailBlaze...  


## **Text preprocessing Function**

In [6]:
# Define a function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Join the tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [7]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf
import pandas as pd
model_name = 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
#Text preprocessing to the input and output columns
data['Input'] = data['Input'].apply(preprocess_text)
data['Output'] = data['Output'].apply(preprocess_text)

# Tokenize the preprocessed input and output sequences
input_sequences = tokenizer(data['Input'].tolist(), padding=True, truncation=True, return_tensors="tf")
output_sequences = tokenizer(data['Output'].tolist(), padding=True, truncation=True, return_tensors="tf")



All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
# Preparation of inputs and labels for training
inputs = {
    "input_ids": input_sequences["input_ids"],
    "attention_mask": input_sequences["attention_mask"],
    "decoder_input_ids": output_sequences["input_ids"][:, :-1],  # remove last token
    "decoder_attention_mask": output_sequences["attention_mask"][:, :-1]  # remove last token
}
labels = output_sequences["input_ids"][:, 1:]  # shift labels to the right

# Compile the model
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.summary()

# Train the model
history = model.fit(inputs, labels, batch_size=2, epochs=10, validation_split=0.2)


Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)
Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
53/53 [==============================] - 173s 787ms/step - loss: 1.3990 - accuracy: 0.7512 - val_loss: 0.5867 - val_accuracy: 0.8925
Epoch 2/10
53/53 [==============================] - 11s 215ms/step - loss: 0.4403 - accuracy: 0.9042 - val_loss: 0.6032 - val_accuracy: 0.8816
Epoch 3/10
53/53 [==============================] - 11s 213ms/step - loss: 0.2845 - accuracy: 0.9297 - val_loss: 0.6100 - val_accuracy: 0.8707
Epoch 4/10
53/53 [==============================] - 13s 242ms/step - loss: 0.1987 - accuracy: 0.9489 - val_loss: 0.6300 - val_accuracy: 0.8860
Epoch 5/10
53/53 [==============================] - 12s 222ms/step - loss: 0.1437 - accuracy: 0.9598 - val_loss: 0.6908 - val_accuracy: 0.8809
Epoch 6/10
53/53 [==============================] - 13s 243ms/step - loss: 0.1425 - accuracy: 0.9633 - val_loss: 0.7336 - val_accuracy: 0.8664
Epoch 7/10
53/

In [9]:
#function of Prediction:
def generate_output(input_text, model, tokenizer):
    # Tokenize the input sequence
    input_sequence = tokenizer(input_text, return_tensors="tf", padding=True, truncation=True)

    # Generate output using the trained model
    generated_output_ids = model.generate(
        input_sequence["input_ids"],
        max_length=100,
        num_beams=5,
        length_penalty=0.6,
        no_repeat_ngram_size=2
    )

    # Decode and return the generated output
    generated_output_text = tokenizer.decode(generated_output_ids[0], skip_special_tokens=True)
    return generated_output_text

In [10]:
#Testing Exemple
new_input_text1 = "StylishWear Inc is a fashion-forward clothing brand established in 2010. Their brand ethos revolves around creativity diversity and inclusivity. StylishWear s product line includes trendy apparel for men wome and children ranging from casual wear to formal attire. The brand's color palette consists of bold and vibrant hues reflecting the dynamic and energetic nature of their designs. StylishWear targets fashion-conscious individuals who seek unique and expressive clothing to complement their personal style."

# Call the function with different inputs
generated_output1 = generate_output(new_input_text1, model, tokenizer)

# Print the generated outputs
print("Generated Output 1:", generated_output1)


Generated Output 1: consists of bold and vibrant hues reflecting the dynamic nature of their clothing. StylishWear focuses fashion-forward individuals seeking unique clothing lines ranging from casual wear to formal attire.


##**3- Text To Speech Model**




In [12]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, set_seed
import torch
import IPython.display as ipd

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

txt = "StylishWear's brand ethos revolves around creativity, diversity, and inclusivity."

inputs = processor(text=txt, return_tensors="pt")
speaker_embeddings = torch.zeros((1, 512))

set_seed(555)  # Make deterministic
speech = model.generate(
    inputs["input_ids"],
    speaker_embeddings=speaker_embeddings,  # Provide speaker embeddings
    vocoder=vocoder,
)

# Play generated audio in Colab
ipd.Audio(speech.squeeze().numpy(), rate=14000)
